In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestRegressor
import joblib
import re
%matplotlib inline

In [2]:
train = pd.read_csv("train_df.csv")
test = pd.read_csv("test_df.csv")
ts_df = pd.read_csv('ts_pred.csv')

In [3]:
pred_date = pd.date_range(start='2021-10-16', end='2021-11-05', freq='1D')
pred_date = pred_date.strftime('%Y-%m-%d').tolist()

forecast_date = pd.date_range(start='2021-11-06', end='2021-12-03', freq='1D')
forecast_date = forecast_date.strftime('%Y-%m-%d').tolist()

pred_forecast_date = pred_date + forecast_date

state_list = ts_df.subregion1_code.unique().tolist()

predictors = ' '.join(train.columns)
mob_predictors = re.findall(r'mobility_\w+', predictors)
searchtrend_predictors = re.findall(r'search_trends_\w+', predictors)
ts_predictors = mob_predictors + searchtrend_predictors

In [4]:
static_latest_raw = pd.DataFrame()
for state in state_list:
    find_df = test[test['subregion1_code'] == state]
    find_df['date'] = pd.to_datetime(find_df['date'])
    date_latest = find_df['date'].max()
    state_row = find_df[find_df['date'] == date_latest]
    static_latest_raw = static_latest_raw.append(state_row)

nj_latest = train[(train['subregion1_code']=='NJ')&(train['date']=='2021-10-04')]
static_latest_raw = static_latest_raw.append(nj_latest)

static_latest = static_latest_raw.drop(["date"]+["cumulative_confirmed"]+ts_predictors, axis = 1)
static_latest = static_latest.reset_index(drop = True)

static_df = pd.DataFrame(np.repeat(static_latest.values, len(pred_forecast_date), axis=0))
static_df.columns = static_latest.columns

static_df = static_df.sort_values(by=['subregion1_code'])

static_df = static_df.reset_index(drop = True)
static_df['date'] = pred_forecast_date * len(state_list)

<ipython-input-4-fe40d575a1f5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  find_df['date'] = pd.to_datetime(find_df['date'])


In [5]:
pred_df = pd.merge(ts_df, static_df, on = ['subregion1_code', 'date'])

pred_df['date'] =pd.to_datetime(pred_df['date'])
pred_df = pred_df.sort_values(by=['subregion1_code', 'date'])
pred_df = pred_df.drop(['type'], axis = 1)

In [6]:
res_df_xgboost = pd.DataFrame()

In [19]:
loaded_xgboost = joblib.load("whole_xgboost.joblib")

In [8]:
cols_when_model_builds = loaded_xgboost.get_booster().feature_names

In [20]:
loaded_xgboost

AttributeError: 'XGBModel' object has no attribute 'enable_categorical'

AttributeError: 'XGBModel' object has no attribute 'enable_categorical'

In [10]:
#for state in static_latest.subregion1_code.unique():
for state in state_list:
    state_col = [state] * (len(pred_date)+len(forecast_date))
    pred_type = ['prediction'] * (len(pred_date))+ ['forecast'] *len(forecast_date)
    pred_curr = pd.DataFrame(list(zip(state_col, pred_date + forecast_date, pred_type)),
                          columns =['subregion1_code', 'date', 'type'])
    

    s_test = pred_df[pred_df["subregion1_code"]==state]
    
    
    s_test = s_test.drop(["date","subregion1_code"], axis = 1)
    s_X_test = s_test.drop(["new_confirmed"], axis = 1)
    
    cols = s_X_test.columns[s_X_test.dtypes.eq('object')]
    s_X_test[cols] = s_X_test[cols].apply(pd.to_numeric, errors='coerce', axis=1)
    
    s_X_test = s_X_test[cols_when_model_builds]
    
    s_y_pred_new = loaded_xgboost.predict(s_X_test)
    
    
    pred_curr['new_confirmed_pred'] = s_y_pred_new
    
    #create prediction dataframe
    res_df_xgboost = res_df_xgboost.append(pred_curr)

KeyError: None

In [11]:
state

'KY'

In [76]:
res_df_xgboost.to_csv('xgboost_pred2.csv', index = False)

In [67]:
res_df_xgboost

,subregion1_code,date,type,new_confirmed_pred
0,AK,2021-10-16,prediction,1653.923096
1,AK,2021-10-17,prediction,1978.842529
2,AK,2021-10-18,prediction,1857.946167
3,AK,2021-10-19,prediction,2620.934570
4,AK,2021-10-20,prediction,2545.250244
...,...,...,...,...
44,WY,2021-11-29,forecast,1819.955933
45,WY,2021-11-30,forecast,1775.355225
46,WY,2021-12-01,forecast,1567.665527
47,WY,2021-12-02,forecast,1886.981689
